In [1]:
!pwd

/notebooks/notebooks/SnowparkRnD/UDFs


In [3]:
from Run_Experiment import run_experiment

In [13]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
import pandas as pd

In [9]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
    }
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

In [11]:
session.add_import("Run_Experiment.py", import_path="Run_Experiment")
session.add_packages("numpy")

In [14]:
@udf
def udf_run_exp(sf_pass: str, dataset: str, target: str) -> pd.DataFrame:
    return run_experiment(sf_pass, dataset, target)

TypeError: the number of arguments (3) is different from the number of argument type hints (0)